In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# Import necessary libraries
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import numpy as np
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


# Step 1: Load the IMDB dataset
dataset = load_dataset('imdb')
train_data = dataset['train']
test_data = dataset['test']

# Step 2: Initialize the BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Step 3: Define a tokenization function
def tokenize_data(data):
    return tokenizer(data['text'], padding='max_length', truncation=True, max_length=128)

# Apply tokenization to the dataset
train_data = train_data.map(tokenize_data, batched=True)
test_data = test_data.map(tokenize_data, batched=True)

# Step 4: Set the format of dataset for PyTorch
train_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Step 5: Load the BERT model for sequence classification
# GPU/CPU 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 모델 정의 후 GPU로 이동
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.to(device)  # 모델을 GPU로 이동
print("GPU available:", torch.cuda.is_available())

# Step 6: Define evaluation metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Step 7: Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Step 8: Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    compute_metrics=compute_metrics
)

# Step 9: Train the model
trainer.train()

# Step 10: Evaluate the model
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)

# Step 11: Save the trained model and tokenizer
model.save_pretrained('./sentiment-analysis-model')
tokenizer.save_pretrained('./sentiment-analysis-tokenizer')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.339000,0.343197,0.868080,0.859492,0.919340,0.806960
2,0.235300,0.418283,0.888480,0.888516,0.888232,0.888800
3,0.112900,0.554280,0.890600,0.891101,0.887039,0.895200


Evaluation results: {'eval_loss': 0.5542803406715393, 'eval_accuracy': 0.8906, 'eval_f1': 0.8911009356957993, 'eval_precision': 0.8870392390011891, 'eval_recall': 0.8952, 'eval_runtime': 184.9544, 'eval_samples_per_second': 135.168, 'eval_steps_per_second': 16.896, 'epoch': 3.0}


('./sentiment-analysis-tokenizer/tokenizer_config.json',
 './sentiment-analysis-tokenizer/special_tokens_map.json',
 './sentiment-analysis-tokenizer/vocab.txt',
 './sentiment-analysis-tokenizer/added_tokens.json',
 './sentiment-analysis-tokenizer/tokenizer.json')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
model.save_pretrained('/content/drive/MyDrive/sentiment-analysis-model')
tokenizer.save_pretrained('/content/drive/MyDrive/sentiment-analysis-tokenizer')


Mounted at /content/drive


('/content/drive/MyDrive/sentiment-analysis-tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/sentiment-analysis-tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/sentiment-analysis-tokenizer/vocab.txt',
 '/content/drive/MyDrive/sentiment-analysis-tokenizer/added_tokens.json',
 '/content/drive/MyDrive/sentiment-analysis-tokenizer/tokenizer.json')

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/sentiment-analysis-model')
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/sentiment-analysis-tokenizer')
trainer.train(resume_from_checkpoint=True)  # Trainer에서 체크포인트 활용


/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3354: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpoint, OPTIMIZER_NAME), map_

Epoch,Training Loss,Validation Loss


TrainOutput(global_step=9375, training_loss=0.0, metrics={'train_runtime': 0.0106, 'train_samples_per_second': 7052253.06, 'train_steps_per_second': 881531.633, 'total_flos': 4933332288000000.0, 'train_loss': 0.0, 'epoch': 3.0})

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks


/content/drive/MyDrive/Colab Notebooks


In [55]:
cd /content/drive/MyDrive/Colab Notebooks/2024R0136COSE47402

/content/drive/MyDrive/Colab Notebooks/2024R0136COSE47402


In [49]:
!git config --global user.email 'ghthd97@naver.com'
!git config --global user.name 'Freerider-song'

In [56]:
!git status

On branch master
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	"HW2/HW2_2017170858_\341\204\200\341\205\265\341\206\267\341\204\222\341\205\251\341\204\211\341\205\251\341\206\274.ipynb"

nothing added to commit but untracked files present (use "git add" to track)


In [31]:
!git restore .
!git clean -f

Removing "HW2/HW2_2017170858_\341\204\200\341\205\265\341\206\267\341\204\222\341\205\251\341\204\211\341\205\251\341\206\274.ipynb"


In [38]:
!git add FinalProject/Final_Project.ipynb
!git commit --amend --no-edit


fatal: pathspec 'FinalProject/Final_Project.ipynb' did not match any files
[master 5734a41] Add FinalProject folder and final_project.py
 Date: Tue Nov 26 14:56:12 2024 +0900
 1 file changed, 73 insertions(+)
 create mode 100644 FinalProject/Final_project.py


In [29]:
!git rebase -i HEAD~3

error: cannot rebase: You have unstaged changes.
error: Please commit or stash them.


In [53]:
!git push origin master --force

Enumerating objects: 30, done.
Counting objects: 100% (30/30), done.
Delta compression using up to 2 threads
Compressing objects: 100% (20/20), done.
Writing objects: 100% (30/30), 5.95 MiB | 12.72 MiB/s, done.
Total 30 (delta 5), reused 29 (delta 5), pack-reused 0
remote: Resolving deltas: 100% (5/5), done.
To https://github.com/Freerider-song/2024R0136COSE47402.git
 + c0ec6b6...5734a41 master -> master (forced update)
